In [ ]:
from pynq import Overlay     # import the overlay
from pynq import allocate    # import for CMA (contingeous memory allocation)
from pynq import DefaultIP   # import the ip connector library for extension
from pynq import Interrupt
from pynq import PL
import asyncio
import numpy as np
import os
import subprocess
import re
import time
import pynq_driver as MY_IP

In [ ]:
# reset and load overlay
PL.reset()
ol = Overlay(f"system.bit")
ol.download()
print("FPGA loaded:", ol.is_loaded())
print("IPs:", ol.ip_dict.keys())

In [ ]:
# check interrupt pin
ol.interrupt_pins
dma = ol.axi_dma_0

In [ ]:
# create an instance of the interrupt
my_interrupt = Interrupt('axi_dma_0/s2mm_introut')

In [ ]:
# Load input from .npy file
input_array  = np.load("inputGenbit.npy").astype(np.float32)
output_array = np.zeros(input_array.shape, dtype=np.float32)

# Allocate physically contiguous memory for input and output
input_buffer = allocate(shape=input_array.shape, dtype=np.float32)
output_buffer = allocate(shape=output_array.shape, dtype=np.float32)

# copy data to input buffer
np.copyto(input_buffer, input_array)

In [ ]:
# get the dma ip and initialize the system
dmaWraper = MY_IP.HLS4ML_IP(dma)
dmaWraper.transfer(input_buffer, output_buffer, my_interrupt, True) # start transfer

In [ ]:
# convert it to numpy array save it to .npy file
outNp = np.array(output_buffer)
np.save("out_hw.npy", outNp)